# Johnson's Algorithm 

In [1]:
import PriorityQ
%run PriorityQ.py
import WeightGraph
%run WeightGraph.py

I'm Priority Queue for Graph Algorithm
I'm Priority Queue for Graph Algorithm


In [2]:
def Dijstra(G,s):
    G.initialize_single_source(s)
    S = []
    
    D = []
    for v in list(G.graph_dict.keys()): 
        D.append((G.nm[v].d, G.nm[v].name))
    Q = priority_queue(D)                   
    #print(Q.h)
    
    while Q.size() != 0: 
        
        u = Q.heap_extract_min()[1]                      # O(VlgV)
        #print(" >> ", u, " vertex 꺼냄")
        S.append(u)
        
        # relaxation을 통해 v.d 를 update 한다.(queue의 값 역시 update 필요)
        # for each vertex v ∈ G.adj[u]
        for v in list(G.graph_dict[u]):                   #
            G.relax(u,v)
            #print(Q.find_index(v), (G.nm[v].d, v))
            Q.key_update(Q.find_index(v),(G.nm[v].d, v))  #  O(ElgV)
        
        #print(Q.h)
        #print(" >> Q's size: ", Q.size())   
    
    #print("shortest path가 결정된 vertices set : ",S)
    # running time : O(( |V| + |E| )lg|V| )

In [3]:
def BellmanFord(G,s):
    srt = G.sort_edge()
    
    G.initialize_single_source('s')
    
    """ Finding the single src shortest parth from s to each vertex"""
    # iteration |V| - 1 
    for i in range(len(G.graph_dict.keys())):           # O(V) * 
        # each edge (u,v) ∈ G.E
        for [u,v] in srt:                               # O(E)
               G.relax(u,v)           
        
    # test whether ∃ negative weight cycle 
    for [u,v] in srt:
        if G.nm[v].d > G.nm[u].d + G.nm[u].edge[v]:
            return False
    
    # T = O(|V||E|)
    return True

In [5]:
import numpy as np 

def Johnson(G):
    #preprocessing
    # making dummy src
    srt = G.sort_edge()
    g2 = G.graph_dict.copy()       # dictionary의 주소를 다르게 copy해야한다.(g2 의 변경이 g1이 변경되지 못하게 한다.)
    w2 = G.weight_dict.copy()
    g2['s'] = set(G.graph_dict.keys())
    w2['s'] = {}
    for v in g2['s']:
        w2['s'][v] = 0
    tempG = Graph(g2,w2)
    
    h = {}
    
    
    # compute BellmanFord algo
    if BellmanFord(tempG,'s') == False:
        print("the input graph contains a negative weight cycle ")
    else:
        for v in list(G.graph_dict.keys()):
            h[v] = tempG.nm[v].d
        for [u,v] in srt:
            # reweighting process (굳히 tempG.edge 에 대해서 바꿀 필요 없다.)
            tempG.nm[u].edge[v] = G.nm[u].edge[v] + h[u] - h[v]
            G.nm[u].edge[v] = tempG.nm[u].edge[v]   ## reweighting processing  w 를 w' 값으로 바꿨다.   
            #print("vertex (", u ,", ",v,") weight를 ", tempG.nm[u].edge[v], "로 변경 ")
        n = len(list(G.graph_dict.keys())) 
        D = np.ones((n,n))

        for u in list(G.graph_dict.keys()):
            Dijstra(G,u) # source를 u vertex로 하여 single src shortest path 구함 v.d 에 값 저장됨
            #print("src ",u," 로 부터의 계산값 !! ")
            #G.showinfo()
            for v in list(G.graph_dict.keys()):
                D[G.nm[v].number][G.nm[u].number] = G.nm[v].d + h[v] - h[u] # comeback reweighting value 
    
    return D 

In [6]:
g = {'a': set(['b','c','e']),
     'b': set(['d','e']),
     'c': set(['b']),
     'd': set(['a','c']),
     'e': set(['d'])}

w = {'a': {'b':3, 'c':8, 'e':-4},
     'b': {'d':1, 'e':7},
     'c': {'b':4},
     'd': {'a':2, 'c':-5},
     'e': {'d':6}}


G = Graph(g,w)

In [7]:
D = Johnson(G)
D

array([[ 0.,  3.,  7.,  2.,  8.],
       [ 1.,  0.,  4., -1.,  5.],
       [-3., -4.,  0., -5.,  1.],
       [ 2.,  1.,  5.,  0.,  6.],
       [-4., -1.,  3., -2.,  0.]])